In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../dataset/meal_service/train.csv')

In [3]:
def get_is_holiday_lists(df):
    # pre-processing    
    is_yesterday_holiday_list = []
    is_tomorrow_holiday_list = []

    for i in range (0, len(df)):
        if df.iloc[i]['요일'] == '월':
            is_yesterday_holiday_list.append(1)
            if i < len(df)-1:
                is_tomorrow_holiday_list.append(0) if df.iloc[i+1]['요일'] == '화' else is_tomorrow_holiday_list.append(1)
            else:
                is_tomorrow_holiday_list.append(0)
        elif df.iloc[i]['요일'] == '화':
            if i < len(df)-1:
                is_tomorrow_holiday_list.append(0) if df.iloc[i+1]['요일'] == '수' else is_tomorrow_holiday_list.append(1)
            else:
                is_tomorrow_holiday_list.append(0)
            if i > 0:
                is_yesterday_holiday_list.append(0) if df.iloc[i-1]['요일'] == '월' else is_yesterday_holiday_list.append(1)
            else:
                is_yesterday_holiday_list.append(0)
        elif df.iloc[i]['요일'] == '수':
            if i < len(df)-1:
                is_tomorrow_holiday_list.append(0) if df.iloc[i+1]['요일'] == '목' else is_tomorrow_holiday_list.append(1)
            else:
                is_tomorrow_holiday_list.append(0)
            if i > 0:
                is_yesterday_holiday_list.append(0) if df.iloc[i-1]['요일'] == '화' else is_yesterday_holiday_list.append(1)
            else:
                is_yesterday_holiday_list.append(0)
        elif df.iloc[i]['요일'] == '목':
            if i < len(df)-1:
                is_tomorrow_holiday_list.append(0) if df.iloc[i+1]['요일'] == '금' else is_tomorrow_holiday_list.append(1)
            else:
                is_tomorrow_holiday_list.append(0)
            if i > 0:
                is_yesterday_holiday_list.append(0) if df.iloc[i-1]['요일'] == '수' else is_yesterday_holiday_list.append(1)
            else:
                is_yesterday_holiday_list.append(0)
        elif df.iloc[i]['요일'] == '금':
            is_tomorrow_holiday_list.append(1)
            if i > 0:
                is_yesterday_holiday_list.append(0) if df.iloc[i-1]['요일'] == '목' else is_yesterday_holiday_list.append(1)
            else:
                is_yesterday_holiday_list.append(0)
                
    return is_yesterday_holiday_list, is_tomorrow_holiday_list

In [4]:
import arrow

def get_is_corona_list(df):
    corona_list = []
    corona_start = arrow.get('2020-03-01')
    for i in range(0, len(df)):
        date = arrow.get(df.iloc[i]['일자'])
        if date >= corona_start:
            corona_list.append(1)
        else:
            corona_list.append(0)
    
    return corona_list

In [5]:
def get_month_list(df):
    month_list = []
    for i in range(0, len(df)):
        month = arrow.get(df.iloc[i]['일자']).month
        month_list.append(month)
    
    return month_list

In [6]:
def seperate_and_processing_menu_str(menu_row):
    # split by space
    splits = menu_row.split(' ')
    menu = []
    is_new_menu = False
    for dish in splits:
        if len(dish) > 1:
            # find (New)
            if '(New)' in dish:
                menu.append(dish.split('(New)')[0] + dish.split('(New)')[1])
                is_new_menu = True
            elif '(' not in dish and ')' not in dish:
                menu.append(dish)
    if len(menu) <= 3:
        # no menu today
        menu = ['-']
    
    return menu, is_new_menu

In [7]:
def get_menu_info(df):
    is_new_lunch_menu = []
    is_new_dinner_menu = []
    for i in range(0, len(df)):
        row_lunch = df.iloc[i]['중식메뉴']
        row_dinner = df.iloc[i]['석식메뉴']
        
        lunch_menu, new_lunch_menu = seperate_and_processing_menu_str(row_lunch)
        is_new_lunch_menu.append(1) if new_lunch_menu else is_new_lunch_menu.append(0)
        
        dinner_menu, new_dinner_menu = seperate_and_processing_menu_str(row_dinner)
        is_new_dinner_menu.append(1) if new_dinner_menu else is_new_dinner_menu.append(0)
        
        # to do 
        # menu classification
    
    return is_new_lunch_menu, is_new_dinner_menu

In [8]:
def get_is_no_dinner_list(df):
    is_no_dinner_list = []
    for i in range(0, len(df)):
        is_no_dinner_list.append(1) if len(df.iloc[i]['석식메뉴']) <= 20 else is_no_dinner_list.append(0)
    
    return is_no_dinner_list

In [9]:
yesterday_holiday_list, tommorow_holiday_list = get_is_holiday_lists(df)
is_corona_list = get_is_corona_list(df)
month_list = get_month_list(df)
is_no_dinner_list = get_is_no_dinner_list(df)
is_new_lunch_list, is_new_dinner_list = get_menu_info(df)

is_yesterday_holiday = pd.DataFrame({'is_yesterday_holiday': yesterday_holiday_list})
is_tomorrow_holiday = pd.DataFrame({'is_tomorrow_holiday': tommorow_holiday_list})
is_corona = pd.DataFrame({'is_corona': is_corona_list})
month = pd.DataFrame({'month': month_list})
no_dinner = pd.DataFrame({'no_dinner': is_no_dinner_list})
is_new_lunch = pd.DataFrame({'is_new_lunch': is_new_lunch_list})
is_new_dinner = pd.DataFrame({'is_new_dinner': is_new_dinner_list})

df = df.join(is_yesterday_holiday)
df = df.join(is_tomorrow_holiday)
df = df.join(is_corona)
df = df.join(month)
df = df.join(no_dinner)
df = df.join(is_new_lunch)
df = df.join(is_new_dinner)

df = df.drop(['일자', '조식메뉴', '중식메뉴', '석식메뉴'],axis='columns')

# 원핫인코딩
df = pd.get_dummies(df, columns=['요일'])

In [10]:
df.head()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,is_yesterday_holiday,is_tomorrow_holiday,is_corona,month,no_dinner,is_new_lunch,is_new_dinner,요일_금,요일_목,요일_수,요일_월,요일_화
0,2601,50,150,238,0.0,1039.0,331.0,1,0,0,2,0,0,0,0,0,0,1,0
1,2601,50,173,319,0.0,867.0,560.0,0,0,0,2,0,0,0,0,0,0,0,1
2,2601,56,180,111,0.0,1017.0,573.0,0,0,0,2,0,0,0,0,0,1,0,0
3,2601,104,220,355,0.0,978.0,525.0,0,0,0,2,0,0,0,0,1,0,0,0
4,2601,278,181,34,0.0,925.0,330.0,0,1,0,2,0,0,0,1,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split

x1_train, x1_test, y1_train, y1_test = train_test_split(df.iloc[:,range(0,len(df))], dt[['중식계']])
x2_train, x2_test, y2_train, y2_test = train_test_split(df.iloc[:,range(0,len(df))], dt[['석식계']])